In [ ]:
import warnings
warnings.filterwarnings('ignore')
import math
import pandas as pd
import numpy as np
import geopandas as gpd
import folium

from xgboost import XGBRegressor
from geopy import distance
from sklearn.model_selection import cross_val_score
from shapely.geometry import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

### Задача:

Прогноз продаж одной из популярных моделей [фичерфонов](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D1%87%D0%B5%D1%80%D1%84%D0%BE%D0%BD) (на картинке ниже пример похожего устройства) в салонах МегаФона
![](https://39.img.avito.st/640x480/8468720439.jpg)

### Исходные данные:

Датасет содержит следующие поля:

1. `point_id` - Индентификатор салона
2. `lon` - Долгота точки
3. `lat` - Широта точки
4. `target` - Значение таргета, усредненное за несколько месяцев и отнормированное

### Требования к решению и советы:

Ниже приведен список из нескольких важных пунктов, необходимых для решения задания. Выполнение каждого из пунктов влияет на итоговую оценку. Вы можете выполнить каждый из пунктов разными способами, самым лучшим будет считаться вариант, когда всё получение и обработка данных будут реализованы на Питоне (пример: вы можете скачать данные из OSM через интерфейс на сайте overpass-turbo или с помощью библиотек `overpass`/`requests`. Оба варианта будут зачтены, но больше баллов можно заработать во втором случае)



1. Салоны расположены в нескольких разных городах, вам необходимо **определить город для каждого салона** (это понадобится во многих частях задания). К этому есть разные подходы. Вы можете провести [обратное геокодирование](https://en.wikipedia.org/wiki/Reverse_geocoding) с помощью геокодера [Nominatim](https://nominatim.org/), доступного через библиотеку `geopy` примерно вот так:
```python
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)
```
В таком случае, вам придется обрабатывать полученную строку адреса, чтобы извлечь название города. Также вы можете скачать из OSM или найти в любом другом источнике границы административно территориальных границ России и пересечь с ними датасет с помощью `geopandas.sjoin` (этот вариант более надежный, но нужно будет разобраться с тем, как устроены границы АТД в OSM, обратите внимание на [этот тег](https://wiki.openstreetmap.org/wiki/Key:admin_level))


2. **Используйте данные OSM**: подумайте, какие объекты могут влиять на продажи фичерфонов. Гипотеза: такие телефоны покупают люди, приезжающие в город или страну ненадолго, чтобы вставить туда отдельную симкарту для роуминга. Можно попробовать использовать местоположения железнодорожных вокзалов (изучите [этот тег](https://wiki.openstreetmap.org/wiki/Tag:railway%3Dstation)). Необходимо использовать хотя бы 5 разных типов объектов из OSM. Скорее всего, вам придется качать данные OSM отдельно для разных городов (см. пример для Нью-Йорка из лекции)


3. **Используйте разные способы генерации признаков**: описать положение салона МегаФона относительно станций метро можно разными способами - найти ***расстояние до ближайшей станции***, или же посчитать, сколько станций попадает в ***500 метровую буферную зону*** вокруг салона. Такие признаки будут нести разную информацию. Так же попробуйте поэкспериментировать с размерами буферных зон (представьте, что значат в реальности радиусы 100, 500, 1000 метров). Попробуйте посчитать расстояние до центра города, до других объектов.

4. **Сделайте визуализации**: постройте 2-3 карты для какого нибудь из городов - как распределен в пространстве таргет, где находятся объекты, полученные вами из OSM. Можете использовать любой инструмент - обычный `plot()`, `folium`, `keplergl`. Если выберете Кеплер, обязательно сохраните в файл конфиг карты, чтобы ее можно было воспроизвести. Сделать это можно вот так:

```python
import json
json_data = kepler_map.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)
```
5. Задание не ограничено приведенными выше пунктами, попробуйте нагенерировать интересных признаков, найти в интернете дополнительные данные (в таком случае в комментарии к коду укажите ссылку на ресурс, откуда взяли данные)



6. Это довольно сложная задача - датасет очень маленький, данные по своей природе довольно случайны. Поэтому место и скор на Kaggle не будут играть решающую роль в оценке, но позволят заработать дополнительные баллы

### Read data

In [ ]:
train = pd.read_csv('data/mf_geo_train.csv')
test = pd.read_csv('data/mf_geo_test.csv')

In [ ]:
admin4 = gpd.read_file("data/admin_level_4.geojson")

In [ ]:
admin6 = gpd.read_file("data/admin_level_6.geojson")

In [ ]:
def pd_to_geo(df):
    df1 = df.reset_index().drop(columns='index')
    geometry = [shapely.geometry.asPoint((row.lon,row.lat)) for index,row in df1.iterrows()]
    df_buf = df1.drop(columns=['lon','lat'])
    result_df = gpd.GeoDataFrame(pd.concat([df_buf,pd.DataFrame({'geometry' : geometry})],axis=1))
    return result_df

In [ ]:
test

In [ ]:
train.head(10)

In [ ]:
pd_to_geo(train).head(10)

In [ ]:
train_df = pd_to_geo(train)

In [ ]:
train_df.shape

In [ ]:
def df_loc(df):
    df_admin6 = gpd.sjoin(pd_to_geo(df),admin6[['name','geometry']],op='within')
    df_admin4 = gpd.sjoin(pd_to_geo(df),admin4[['name','geometry']])
    df_loc = df_admin6.append(df_admin4.drop(index=df_admin6.index)).reset_index().drop(columns=['index','index_right']).rename(columns={'name':'location'})
    return df_loc

In [ ]:
def df_shape(df):
    geometry_100 = []
    geometry_500 = []
    geometry_1000 = []
    for index,x in df.iterrows():
        geometry_100.append(geodesic_point_buffer(x.lon,x.lat,0.1))
        geometry_500.append(geodesic_point_buffer(x.lon,x.lat,0.5))
        geometry_1000.append(geodesic_point_buffer(x.lon,x.lat,1))
    df_shape = pd.concat([df,pd.DataFrame({'100':geometry_100}),pd.DataFrame({'500':geometry_500}),pd.DataFrame({'1000':geometry_1000})],axis=1)
    return df_shape

In [ ]:
df_shape(test)

In [ ]:
train_loc = df_loc(train)

In [ ]:
train_loc

In [ ]:
test_loc = df_loc(test)

In [ ]:
train_loc['location'].value_counts()

In [ ]:
m = folium.Map(location=[55.772910,37.678790],zoom_start=8)
folium.GeoJson(admin5[['name','geometry']][admin5.name.isin(train_loc.location)].geometry).add_to(m)
folium.GeoJson(admin4[['name','geometry']][admin4.name.isin(train_loc.location)].geometry).add_to(m)
m

# Features generating

In [ ]:

for index,row in train.iterrows():
    folium.Marker([row.lat,row.lon],popup=row.target).add_to(m)
m

In [ ]:
from functools import partial
import pyproj
from shapely.ops import transform
import requests
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')

def geodesic_point_buffer(lon, lat, km):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lon=lon, lat=lat)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf)

In [ ]:
def city_comp(name):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = '''
    [out:json];
    (area["name"="{}"];) -> .search;
    (node["shop"="mobile_phone"](area.search);) -> .node;
    .node out geom;
    '''.format(name)

    response = requests.get(overpass_url, 
                        params={'data': overpass_query})
    amenity = response.json()
    amenity_osm = pd.DataFrame(amenity['elements'])
    amenity_osm = amenity_osm.join(
        pd.DataFrame([x['tags'] for x in amenity['elements']])).drop('tags', axis=1)
    try:
        comp = amenity_osm.name.value_counts().head(6).drop(index=['МегаФон','Мегафон']).index
    except: 
        try: 
            comp = amenity_osm.name.value_counts().head(6).drop(index=['МегаФон']).index
        except:
            try:
                comp = amenity_osm.name.value_counts().head(6).drop(index=['Мегафон']).index
            except:
                comp = amenity_osm.name.value_counts().head(6).index
    comp_sal = amenity_osm[amenity_osm[['name']].isin(comp).values]
    
    return comp_sal.dropna(axis=1)[['lat','lon','name']]

In [ ]:
def df_comp(df):
    df1 = pd.DataFrame()
    for i in df.location.value_counts().index:
        print(i)
        df1 = pd.concat([df1,city_comp(i)],axis=0)
    return df1

In [ ]:
df_comp(df_loc(train))

In [ ]:
def df100(df):
    df100 = gpd.sjoin(
        gpd.GeoDataFrame(
            df_shape(df).rename(columns={'100':'geometry'})),
            pd_to_geo(df_comp(df_loc(df))[['lat','lon','name']]),how='left')
    df_2f_100 = pd.concat(
        [df_loc(df).drop(columns='target').set_index('point_id'),
         df100.groupby('point_id')['name'].count()],axis=1).reset_index().rename(columns={'name':'salons'}) 
    X = df_2f_100.set_index('index').drop(columns='geometry')
    map1 = dict(zip(X.location.value_counts().index,[x for x in range(0,len(X.location.value_counts().index))]))
    X.location = X.location.map(map1)
    return X

In [ ]:
def df500(df):
    df100 = gpd.sjoin(
        gpd.GeoDataFrame(
            df_shape(df).rename(columns={'500':'geometry'})),
            pd_to_geo(df_comp(df_loc(df))[['lat','lon','name']]),how='left')
    df_2f_100 = pd.concat(
        [df_loc(df).drop(columns='target').set_index('point_id'),
         df100.groupby('point_id')['name'].count()],axis=1).reset_index().rename(columns={'name':'salons'}) 
    X = df_2f_100.set_index('index').drop(columns='geometry')
    map1 = dict(zip(X.location.value_counts().index,[x for x in range(0,len(X.location.value_counts().index))]))
    X.location = X.location.map(map1)
    return X

In [ ]:
def city_station(name):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = '''
    [out:json];
    (area["name"="{}"];) -> .search;
    (node["railway"="station"](area.search);) -> .node;
    .node out geom;
    '''.format(name)

    response = requests.get(overpass_url, 
                        params={'data': overpass_query})
    amenity = response.json()
    amenity_osm = pd.DataFrame(amenity['elements'])
    amenity_osm = amenity_osm.join(
        pd.DataFrame([x['tags'] for x in amenity['elements']])).drop('tags', axis=1)
    
    
    return amenity_osm[['lat','lon','name']]

In [ ]:
m = folium.Map(location=[55.772910,37.678790],zoom_start=8)
for index, row in city_station('Москва').iterrows():
    folium.Marker([row.lat,row.lon],icon=folium.Icon(color='green')).add_to(m)
m

In [ ]:
def dist(df,st):
    gst = pd_to_geo(st)
    dist1 = []
    for point in df.point_id:
        a = []
        point_coord = df[df.point_id==point].geometry.values.y, df[df.point_id==point].geometry.values.x
        for index,p in gst.iterrows():
            station_coord = p.geometry.y,p.geometry.x
            a.append(distance.geodesic(point_coord,station_coord).m)
        dist1.append(np.array(a).min())

    return(dist1)

In [ ]:
def df_dis(df):
    df_dis1 = df_loc(df).copy()
    for i, x in enumerate(df_dis1.location.value_counts().index):
         df_dis1.loc[df_dis1.location == x, 'railway_dist'] = dist(df_dis1[df_dis1.location == x],city_station(x))
    return df_dis1

In [ ]:
df_dis(test)

In [ ]:
def df_sta(df):
    df1 = pd.DataFrame()
    for i in df.location.value_counts().index:
        print(i)
        df1 = pd.concat([df1,city_station(i)],axis=0)
    return df1

In [ ]:
def df100_2(df):
    df100 = gpd.sjoin(
        gpd.GeoDataFrame(
            df_shape(df).rename(columns={'100':'geometry'})),
            pd_to_geo(df_sta(df_loc(df))[['lat','lon','name']]),how='left')
    df_2f_100 = pd.concat(
        [df_loc(df).drop(columns='target').set_index('point_id'),
         df100.groupby('point_id')['name'].count()],axis=1).reset_index().rename(columns={'name':'stations'}) 
    X = df_2f_100.set_index('index').drop(columns='geometry')
    map1 = dict(zip(X.location.value_counts().index,[x for x in range(0,len(X.location.value_counts().index))]))
    X.location = X.location.map(map1)
    return X

In [ ]:
def df500_2(df):
    df100 = gpd.sjoin(
        gpd.GeoDataFrame(
            df_shape(df).rename(columns={'500':'geometry'})),
            pd_to_geo(df_sta(df_loc(df))[['lat','lon','name']]),how='left')
    df_2f_100 = pd.concat(
        [df_loc(df).drop(columns='target').set_index('point_id'),
         df100.groupby('point_id')['name'].count()],axis=1).reset_index().rename(columns={'name':'stations'}) 
    X = df_2f_100.set_index('index').drop(columns='geometry')
    map1 = dict(zip(X.location.value_counts().index,[x for x in range(0,len(X.location.value_counts().index))]))
    X.location = X.location.map(map1)

    return X

In [ ]:
def df1000_2(df):
    df100 = gpd.sjoin(
        gpd.GeoDataFrame(
            df_shape(df).rename(columns={'1000':'geometry'})),
            pd_to_geo(df_sta(df_loc(df))[['lat','lon','name']]),how='left')
    df_2f_100 = pd.concat(
        [df_loc(df).drop(columns='target').set_index('point_id'),
         df100.groupby('point_id')['name'].count()],axis=1).reset_index().rename(columns={'name':'stations'}) 
    X = df_2f_100.set_index('index').drop(columns='geometry')
    map1 = dict(zip(X.location.value_counts().index,[x for x in range(0,len(X.location.value_counts().index))]))
    X.location = X.location.map(map1)
    return X

In [ ]:
df100_2(train)

In [ ]:
df500_2(train)[0].stations.value_counts()

In [ ]:
df1000_2(train)[0].stations.value_counts()

In [ ]:
for index,x in df_shape(train).iterrows():
    folium.GeoJson(x['1000'],style_function=lambda x :{'fillColor':'red'}).add_to(m)
    folium.GeoJson(x['500'],style_function=lambda x :{'fillColor':'black'}).add_to(m)
    folium.GeoJson(x['100']).add_to(m)
m

In [ ]:
def city_mall(name):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = '''
    [out:json];
    (area["name"="{}"];) -> .search;
    (node["shop"="mall"](area.search);) -> .node;
    .node out count;
    .node out geom;
    '''.format(name)

    response = requests.get(overpass_url, 
                        params={'data': overpass_query})
    amenity = response.json()
    amenity_osm = pd.DataFrame(amenity['elements'])
    amenity_osm = amenity_osm.join(
        pd.DataFrame([x['tags'] for x in amenity['elements']])).drop('tags', axis=1)
    try:
        comp = amenity_osm.name.value_counts().head(6).drop(index=['МегаФон','Мегафон']).index
    except: 
        try: 
            comp = amenity_osm.name.value_counts().head(6).drop(index=['МегаФон']).index
        except:
            try:
                comp = amenity_osm.name.value_counts().head(6).drop(index=['Мегафон']).index
            except:
                comp = amenity_osm.name.value_counts().head(6).index
    comp_sal = amenity_osm[amenity_osm[['name']].isin(comp).values]
    
    return comp_sal.dropna(axis=1)[['lat','lon','name']]

In [ ]:
def df_mall(df):
    df1 = pd.DataFrame()
    for i in df.location.value_counts().index:
        print(i)
        df1 = pd.concat([df1,city_mall(i)],axis=0)
    return df1

In [ ]:
def df100_3(df):
    df100 = gpd.sjoin(
        gpd.GeoDataFrame(
            df_shape(df).rename(columns={'1000':'geometry'})),
            pd_to_geo(df_mall(df_loc(df))[['lat','lon','name']]),how='left')
    df_2f_100 = pd.concat(
        [df_loc(df).drop(columns='target').set_index('point_id'),
         df100.groupby('point_id')['name'].count()],axis=1).reset_index().rename(columns={'name':'malls'}) 
    X = df_2f_100.set_index('index').drop(columns='geometry')
    map1 = dict(zip(X.location.value_counts().index,[x for x in range(0,len(X.location.value_counts().index))]))
    X.location = X.location.map(map1)

    return X

In [ ]:
df100_3(train)

In [ ]:
def city_sleep(name):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = '''
    [out:json];
    (area["name"="{}"];) -> .search;
    (node["amenity"="school"](area.search);) -> .node1;
    (node["amenity"="hospital"](area.search);) -> .node2;
    (node["leisure"="playground"](area.search);) -> .node;
    .node out geom;
    .node1 out geom;
    .node2 out geom;
    '''.format(name)

    response = requests.get(overpass_url, 
                        params={'data': overpass_query})
    amenity = response.json()
    amenity_osm = pd.DataFrame(amenity['elements'])
    amenity_osm = amenity_osm.join(
        pd.DataFrame([x['tags'] for x in amenity['elements']])).drop('tags', axis=1)
    
    return amenity_osm

In [ ]:
def df_sleep(df):
    df1 = pd.DataFrame()
    for i in df.location.value_counts().index:
        print(i)
        df1 = pd.concat([df1,city_sleep(i)],axis=0)
    return df1

In [ ]:
def df1000_4(df):
    df100 = gpd.sjoin(
        gpd.GeoDataFrame(
            df_shape(df).rename(columns={'1000':'geometry'})),
            pd_to_geo(df_sleep(df_loc(df))[['lat','lon','id']]),how='left')
    df_2f_100 = pd.concat(
        [df_loc(df).drop(columns='target').set_index('point_id'),
         df100.groupby('point_id')['id'].count()],axis=1).reset_index().rename(columns={'id':'infrastructure'}) 
    X = df_2f_100.set_index('index').drop(columns='geometry')
    map1 = dict(zip(X.location.value_counts().index,[x for x in range(0,len(X.location.value_counts().index))]))
    X.location = X.location.map(map1)

    return X

In [ ]:
salons = df100(train)

In [ ]:
stations = df500_2(train)

In [ ]:
malls = df100_3(train)

In [ ]:
sleep = df1000_4(train)

In [ ]:
distant = df_dis(train)

In [ ]:
distant.drop(columns=['target','geometry','location']).set_index('point_id')

In [ ]:
train_df = pd.concat([sleep,malls.drop(columns='location'),stations.drop(columns='location'),salons.drop(columns='location'),
                     distant.drop(columns=['target','geometry','location']).set_index('point_id')],axis=1)

In [ ]:
train_df

In [ ]:
salons_test = df100(test)

In [ ]:
stations_test = df500_2(test)

In [ ]:
malls_test = df100_3(test)

In [ ]:
sleep_test = df1000_4(test)

In [ ]:
dist_test = df_dis(test)

In [ ]:
dist_test.drop(columns=['target','geometry','location']).set_index('point_id')

In [ ]:
test_df = pd.concat([sleep_test,malls_test.drop(columns='location'),stations_test.drop(columns='location'),salons_test.drop(columns='location'),
                    dist_test.drop(columns=['target','geometry','location']).set_index('point_id')],axis=1)

In [ ]:
y = train.drop(columns=['lon','lat']).set_index('point_id')

In [ ]:
test_df

### Fit model

In [ ]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

In [ ]:
xg = XGBRegressor()

In [ ]:
rf = RandomForestRegressor()

In [ ]:
gb = GradientBoostingRegressor(learning_rate=0.01,max_depth=2)

In [ ]:
model = GradientBoostingRegressor(learning_rate=0.01,max_depth=2).fit(train_df, y)

In [ ]:
-1*cross_val_score(xg,train_df,y=y,cv=20,scoring='neg_mean_absolute_error').mean()

In [ ]:
-1*cross_val_score(gb,train_df,y=y,cv=20,scoring='neg_mean_absolute_error').mean()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df, y)
model = LinearRegression().fit(train_df, y)

In [ ]:
cross_val_score(model,train_df,y,cv=20,scoring='neg_mean_absolute_error').mean()

In [ ]:
mean_absolute_error(y_valid, model.predict(X_valid))

In [ ]:
model = LinearRegression().fit(train_df, y)

In [ ]:
model.predict(test_df)

### Make submission

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission['target'] = model.predict(test_df)
submission.to_csv('data/my_submission_01.csv', index=False)